In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import metrics
from sklearn import decomposition
from sklearn import manifold
from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy
import random
import time

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

/home/jukie/anaconda3/envs/py37/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('/home/jukie/Project/eeg-eye-state-classification-master/eeg_eye_state.csv', skiprows = [i for i in range(19)], header=None)

In [3]:
X = df.loc[:,0:13]
y = df.loc[:,14:]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15
#, random_state=1234
)

In [5]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [6]:
print(f"Training Data has {len(X_train)} examples")
print(f"Testing Data has {len(X_test)} examples")

Training Data has 12717 examples
Testing Data has 2245 examples


In [7]:
y_train = y_train.to_numpy()


In [8]:
y_test =y_test.to_numpy()

In [9]:
## Train Data
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


train_data = trainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(y_train))
## Test Data    
class testData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

test_data = testData(torch.FloatTensor(X_test),
                    torch.FloatTensor(y_test))

In [10]:
class accData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

acc_data = accData(torch.FloatTensor(X_test))

In [11]:
# Create DataLoaders for torch
BATCH_SIZE = 64
LEARNING_RATE=0.001
EPOCHS=100

train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE,)
acc_loader = DataLoader(dataset=acc_data, batch_size=1)

In [12]:
class eca_layer(nn.Module):
    """Constructs a ECA module.

    Args:
        channel: Number of channels of the input feature map
        k_size: Adaptive selection of kernel size
    """
    def __init__(self, channel, k_size=3):
        super(eca_layer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv = nn.Conv1d(1, 1, kernel_size=k_size, padding=(k_size - 1) // 2, bias=False) 
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # feature descriptor on the global spatial information
        y = self.avg_pool(x)

        # Two different branches of ECA module
        y = self.conv(y.squeeze(-1).transpose(-1, -2)).transpose(-1, -2).unsqueeze(-1)

        # Multi-scale information fusion
        y = self.sigmoid(y)

        return x * y.expand_as(x)

In [13]:
class MLPNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.dim = nn.Linear(1, 64)
        self.layer_1 = nn.Linear(64, 128) 
        self.layer_2 = nn.Linear(128, 128)
        self.eca1 = eca_layer(64,3)
        self.eca2 = eca_layer(256,3)
        self.flat = nn.Sequential(
            nn.Linear(1792,128),
            #nn.Dropout(p=0.5),
            nn.ReLU())
        self.layer_out = nn.Linear(128, 1)
        
        self.leakyRelu = nn.LeakyReLU()
        self.dropout = nn.Dropout(p=0.25)
        self.batchnorm1 = nn.BatchNorm1d(14)
        self.batchnorm2 = nn.BatchNorm1d(14)

    def forward(self, x):
        x = torch.unsqueeze(x,2)
        x = self.dim(x)
        x = self.eca1(x)
        x = self.leakyRelu(self.layer_1(x))
        x = self.batchnorm1(x)
        x = self.dropout(x)
        x = self.leakyRelu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.eca2(x)
        x = self.dropout(x)
        x = torch.flatten(x,1)
        x = self.flat(x)
        x = self.layer_out(x)
        return x

model = MLPNetwork()

In [14]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 254,655 trainable parameters


In [15]:
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=5e-4)
criterion = nn.BCEWithLogitsLoss()
device = torch.device('cuda:0')

In [16]:
model = model.to(device)
criterion = criterion.to(device)

In [17]:
def accuracy(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [27]:
#model.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    tepoch_loss = 0
    tepoch_acc = 0
    model.train()
    for X_batch, y_batch in train_loader:
        
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        acc = accuracy(y_pred, y_batch)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    model.eval()
    for X_batch, y_batch in test_loader:
        
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        
        y_pred = model(X_batch)
        tloss = criterion(y_pred, y_batch)
        tacc = accuracy(y_pred, y_batch)
                
        tepoch_loss += tloss.item()
        tepoch_acc += tacc.item()
        
    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')    
    print(f'Epoch {e+0:03}: | tLoss: {tepoch_loss/len(test_loader):.5f} | tAcc: {tepoch_acc/len(test_loader):.3f}')

Epoch 001: | Loss: 0.46582 | Acc: 76.920
Epoch 001: | tLoss: 0.60830 | tAcc: 68.472
Epoch 002: | Loss: 0.47231 | Acc: 76.824
Epoch 002: | tLoss: 0.59249 | tAcc: 67.556
Epoch 003: | Loss: 0.46509 | Acc: 77.271
Epoch 003: | tLoss: 0.66690 | tAcc: 68.917
Epoch 004: | Loss: 0.46573 | Acc: 77.437
Epoch 004: | tLoss: 0.56636 | tAcc: 71.083
Epoch 005: | Loss: 0.47182 | Acc: 76.920
Epoch 005: | tLoss: 0.68242 | tAcc: 67.444
Epoch 006: | Loss: 0.47343 | Acc: 76.693
Epoch 006: | tLoss: 0.71469 | tAcc: 65.000
Epoch 007: | Loss: 0.47405 | Acc: 76.874
Epoch 007: | tLoss: 0.61668 | tAcc: 71.528
Epoch 008: | Loss: 0.46992 | Acc: 77.005
Epoch 008: | tLoss: 0.58115 | tAcc: 69.000
Epoch 009: | Loss: 0.46998 | Acc: 77.035
Epoch 009: | tLoss: 0.65384 | tAcc: 65.139
Epoch 010: | Loss: 0.47020 | Acc: 77.246
Epoch 010: | tLoss: 0.69493 | tAcc: 68.889
Epoch 011: | Loss: 0.46955 | Acc: 77.040
Epoch 011: | tLoss: 0.69307 | tAcc: 68.861
Epoch 012: | Loss: 0.47155 | Acc: 76.874
Epoch 012: | tLoss: 0.60715 | tAcc:

In [ ]:
#model.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    tepoch_loss = 0
    tepoch_acc = 0
    model.train()
    for X_batch, y_batch in train_loader:
        
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        acc = accuracy(y_pred, y_batch)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    model.eval()
    for X_batch, y_batch in test_loader:
        
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        
        y_pred = model(X_batch)
        tloss = criterion(y_pred, y_batch)
        tacc = accuracy(y_pred, y_batch)
                
        tepoch_loss += tloss.item()
        tepoch_acc += tacc.item()
        
    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')    
    print(f'Epoch {e+0:03}: | tLoss: {tepoch_loss/len(test_loader):.5f} | tAcc: {tepoch_acc/len(test_loader):.3f}')

Epoch 001: | Loss: 0.63653 | Acc: 62.591
Epoch 001: | tLoss: 0.87253 | tAcc: 58.305
Epoch 002: | Loss: 0.57491 | Acc: 69.778
Epoch 002: | tLoss: 0.78046 | tAcc: 60.525
Epoch 003: | Loss: 0.53977 | Acc: 71.847
Epoch 003: | tLoss: 0.75252 | tAcc: 60.339
Epoch 004: | Loss: 0.51897 | Acc: 74.097
Epoch 004: | tLoss: 0.79355 | tAcc: 60.983
Epoch 005: | Loss: 0.49895 | Acc: 75.415
Epoch 005: | tLoss: 1.58813 | tAcc: 58.508
Epoch 006: | Loss: 0.48755 | Acc: 76.097
Epoch 006: | tLoss: 0.80177 | tAcc: 62.559
Epoch 007: | Loss: 0.46786 | Acc: 77.659
Epoch 007: | tLoss: 0.85773 | tAcc: 65.203
Epoch 008: | Loss: 0.44878 | Acc: 78.330
Epoch 008: | tLoss: 1.22177 | tAcc: 62.051
Epoch 009: | Loss: 0.44350 | Acc: 78.540
Epoch 009: | tLoss: 1.62378 | tAcc: 61.271
Epoch 010: | Loss: 0.43550 | Acc: 79.773
Epoch 010: | tLoss: 1.14529 | tAcc: 60.797
Epoch 011: | Loss: 0.42088 | Acc: 80.216
Epoch 011: | tLoss: 0.98188 | tAcc: 62.000
Epoch 012: | Loss: 0.42173 | Acc: 80.182
Epoch 012: | tLoss: 1.77694 | tAcc:

In [25]:
y_pred_list = []
model.eval()
with torch.no_grad():
    for X_batch in acc_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [23]:
#加了eca特征维度扩张bp
print(classification_report(y_test, y_pred_list))

              precision    recall  f1-score   support

           0       0.74      0.72      0.73      1233
           1       0.67      0.70      0.68      1012

    accuracy                           0.71      2245
   macro avg       0.71      0.71      0.71      2245
weighted avg       0.71      0.71      0.71      2245



In [26]:
#加了eca特征维度扩张bp
print(classification_report(y_test, y_pred_list))

              precision    recall  f1-score   support

           0       0.74      0.72      0.73      1233
           1       0.67      0.69      0.68      1012

    accuracy                           0.71      2245
   macro avg       0.70      0.70      0.70      2245
weighted avg       0.71      0.71      0.71      2245



In [ ]:
#特征维度扩张bp
print(classification_report(y_test, y_pred_list))

              precision    recall  f1-score   support

           0       0.71      0.73      0.72      1233
           1       0.66      0.63      0.65      1012

    accuracy                           0.69      2245
   macro avg       0.69      0.68      0.68      2245
weighted avg       0.69      0.69      0.69      2245

